In [14]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2


# if cannot import the modules, add the parent directory to system path might help

import os, tqdm, sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/'
sys.path.append(parent_dir)

from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.words import make_corpus_0, get_wordcount_list
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import inflect
from collections import Counter
import spacy
import copy
random_seed = 2019

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 17.4 µs
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 36.9 ms


In [15]:
dic = load(dir_save = '../big_data/dic_20190830.pickle')
ls = [i for i,v in dic.items() if len(v['ingredients'])>1]
print('drop %d recipes with less than 2 ingredients' %(len(dic)-len(ls)))
ls = [i for i in ls if len(dic[i]['directions'])>1]
print('furthur drop %d recipes with less than 2 instructions' %(len(dic)-len(ls)))
desc = [i for i in ls if len(dic[i]['description'])<1]
print('drop %d recipes with no description' %(len(desc)))
print('now we are using recipe54k %d' % len(ls))

exist
drop 46 recipes with less than 2 ingredients
furthur drop 1026 recipes with less than 2 instructions
drop 0 recipes with no description
now we are using recipe54k 54076
time: 2.59 s


In [16]:
### example of spacy
nlp = spacy.load('en_core_web_lg')

directions = ['preheat oven to 350 degrees f ( 175 degrees c ) .',
 'line a baking sheet with a sheet of aluminum foil .',
 'stir together ketchup and mustard on a plate until mixed .',
 'place the cornflake crumbs in a shallow bowl .',
 'roll each hot dog in the ketchup mixture , then roll in the cornflake crumbs to coat .',
 'place onto prepared baking sheet .',
 'bake in preheated oven until the hot dogs are hot on the inside , and crispy on the outside , 15 to 20 minutes .']

for sent in directions:
    doc = nlp(sent)
    for token in doc:
        if token.pos_ == 'VERB':
            print(token.text, token.lemma_, token.pos_, token.tag_)
    print('-'*30)
    for chunk in doc.noun_chunks:
        root_lemma = [token.lemma_ for token in doc if token.text == chunk.root.text][0]
        print(chunk.text,'|', chunk.root.text,'|', root_lemma)
        
    print('='*30)

preheat preheat VERB VB
------------------------------
oven | oven | oven
350 degrees | degrees | degree
f ( 175 degrees c | c | c
line line VERB VB
------------------------------
a baking sheet | sheet | sheet
a sheet | sheet | sheet
aluminum foil | foil | foil
stir stir VERB VB
------------------------------
ketchup | ketchup | ketchup
mustard | mustard | mustard
a plate | plate | plate
place place VERB VB
------------------------------
the cornflake crumbs | crumbs | crumb
a shallow bowl | bowl | bowl
roll roll VERB VB
roll roll VERB VB
------------------------------
each hot dog | dog | dog
the ketchup mixture | mixture | mixture
the cornflake crumbs | crumbs | crumb
coat | coat | coat
------------------------------
place | place | place
prepared baking sheet | sheet | sheet
bake bake VERB VB
are be VERB VBP
crispy crispy VERB VBN
------------------------------
preheated oven | oven | oven
the hot dogs | dogs | dog
the inside | inside | inside
the outside | outside | outside
15 to 

### goal 1 : calculate precision and recall of verb and noun with spacy
### goal 2 : further examine the precision and recall for ingredients and non-ingredients (via NY time parser)

In [17]:
class spacy_single_sent:
    def __init__(self, sent):
        '''
        Args: sent: string
        '''
        self.doc = nlp(sent)
    def noun(self):
        noun = []
        for chunk in self.doc.noun_chunks:
            noun.append([token.lemma_ for i, token in enumerate(self.doc) if i == chunk.end-1][0])
            #noun.append([token.text for i, token in enumerate(self.doc) if i == chunk.end-1][0]) # show original text
        return noun
    def verb(self):
        return [token.lemma_ for token in self.doc if token.pos_ == 'VERB']

def spacy_sentences(directions):
    '''
    Args: directions: list of strings
    '''
    nouns, verbs = [], []
    for sent in directions:
        read_sent = spacy_single_sent(sent)
        nouns += read_sent.noun()
        verbs += read_sent.verb()
    return nouns, verbs

class metrics:
    def __init__(self, list_true, list_pred):
        self.y_pred = list_pred
        self.y_true = list_true
        
    # frequency weighted
    def f1_freq(self):
        precision = self.precision_freq()
        recall = self.recall_freq()
        return precision*recall/ 2*(precision + recall)
    def precision_freq(self):
        return self.scoring(self.y_pred, self.y_true)
    def recall_freq(self):
        return self.scoring(self.y_true, self.y_pred)
    def scoring(self, n1, n2):
        n1c, n2c = copy.deepcopy(n1), copy.deepcopy(n2)
        score = 0
        for word in n1c:
            if word in n2c:
                score +=1
                n2c.remove(word)
        if len(n1c): 
            return score/len(n1c)
    # without frequency weighted
    def precision(self):
        if len(self.y_pred): 
            return len(set(self.y_true) & set(self.y_pred))/len(set(self.y_pred))
    def recall(self):
        if len(self.y_true):
            return len(set(self.y_true) & set(self.y_pred))/len(set(self.y_true))
    def f1(self):
        precision = self.precision()
        recall = self.recall()
        return precision*recall/ 2*(precision + recall)

time: 33.4 ms


In [24]:
filename = '../../to_gpt2/generation_333k_sorted/'
def load_dir_data(filename):
    ls = []
    if os.path.isdir(filename):
        # Directory
        for (dirpath, _, fnames) in os.walk(filename):
            for fname in fnames:
                path = os.path.join(dirpath, fname)
                with open(path, 'r') as fp:
                    raw_text = fp.read()
                # if it contain instr
                if fname[-5] == 'd':
                    dic[int(fname[:-5])]['generated_instr'] = raw_text.split('.')
                    ls.append(int(fname[:-5]))# only interested in instr
                # if it contain ingred
                if fname[-5] == 'i':
                    dic[int(fname[:-5])]['generated_ingred'] = raw_text.split('$')
                # if it contain name
                if fname[-5] == 't':
                    dic[int(fname[:-5])]['generated_name'] = raw_text
                    
    return sorted(list(set(ls)))
ls = load_dir_data(filename)

time: 110 ms


In [25]:
for i, v in tqdm.tqdm(dic.items()):
    if i in ls:
        n_pred, v_pred = spacy_sentences(dic[i]['generated_instr'])
        n_true, v_true = spacy_sentences(dic[i]['directions'])
        score = metrics(n_true, n_pred)
        scores = {'noun_recall':score.recall(), 'noun_recall_f':score.recall_freq(),
                  'noun_precision':score.precision(), 'noun_precision_f':score.precision_freq(),
                 }
        dic[i].update(scores)
        score = metrics(v_true, v_pred)
        scores = {'verb_recall':score.recall(), 'verb_recall_f':score.recall_freq(),
                  'verb_precision':score.precision(), 'verb_precision_f':score.precision_freq(),
                 }
        dic[i].update(scores)
        
df2 = pd.DataFrame.from_dict(dic, orient = 'index')
df2[[col for col in df2.columns if 'recall' in col or 'precision' in col]].iloc[ls].mean()

100%|██████████| 55102/55102 [01:39<00:00, 554.39it/s]


noun_recall         0.491854
noun_recall_f       0.445608
noun_precision      0.606065
noun_precision_f    0.521618
verb_recall         0.312607
verb_recall_f       0.291760
verb_precision      0.372829
verb_precision_f    0.337052
dtype: float64

time: 1min 42s


In [31]:
filename = '../../to_gpt2/generation_333k/'
ls = load_dir_data(filename)

time: 87.2 ms


In [32]:
len(ls)

510

time: 23.6 ms


In [ ]:
# batch = 8 top-k sampling (k=3)
filename = '../../to_gpt2/generation_333k/'
ls = load_dir_data(filename)
for i, v in tqdm.tqdm(dic.items()):
    if i in ls:
        n_pred, v_pred = spacy_sentences(dic[i]['generated_instr'])
        n_true, v_true = spacy_sentences(dic[i]['directions'])
        score = metrics(n_true, n_pred)
        scores = {'noun_recall':score.recall(), 'noun_recall_f':score.recall_freq(),
                  'noun_precision':score.precision(), 'noun_precision_f':score.precision_freq(),
                 }
        dic[i].update(scores)
        score = metrics(v_true, v_pred)
        scores = {'verb_recall':score.recall(), 'verb_recall_f':score.recall_freq(),
                  'verb_precision':score.precision(), 'verb_precision_f':score.precision_freq(),
                 }
        dic[i].update(scores)
        
df2 = pd.DataFrame.from_dict(dic, orient = 'index')
df2[[col for col in df2.columns if 'recall' in col or 'precision' in col]].iloc[ls].mean()

In [8]:
# batch = 8 top-k sampling (k=3) verb *5
# batch = 8 version
filename = '../../to_gpt2/generation_358k/'
ls = load_dir_data(filename)
for i, v in tqdm.tqdm(dic.items()):
    if i in ls:
        n_pred, v_pred = spacy_sentences(dic[i]['generated_instr'])
        n_true, v_true = spacy_sentences(dic[i]['directions'])
        score = metrics(n_true, n_pred)
        scores = {'noun_recall':score.recall(), 'noun_recall_f':score.recall_freq(),
                  'noun_precision':score.precision(), 'noun_precision_f':score.precision_freq(),
                 }
        dic[i].update(scores)
        score = metrics(v_true, v_pred)
        scores = {'verb_recall':score.recall(), 'verb_recall_f':score.recall_freq(),
                  'verb_precision':score.precision(), 'verb_precision_f':score.precision_freq(),
                 }
        dic[i].update(scores)
        
df2 = pd.DataFrame.from_dict(dic, orient = 'index')
df2[[col for col in df2.columns if 'recall' in col or 'precision' in col]].iloc[ls].mean()

100%|██████████| 55102/55102 [01:42<00:00, 538.10it/s]


noun_recall         0.453282
noun_recall_f       0.412267
noun_precision      0.587857
noun_precision_f    0.507792
verb_recall         0.330381
verb_recall_f       0.314204
verb_precision      0.372977
verb_precision_f    0.336897
dtype: float64

time: 1min 45s


In [7]:
# i-prone version (old but has a high F1)
filename = '../../to_gpt2/generation_1221k_topk/'
ls = load_dir_data(filename)
for i, v in tqdm.tqdm(dic.items()):
    if i in ls:
        n_pred, v_pred = spacy_sentences(dic[i]['generated_instr'])
        n_true, v_true = spacy_sentences(dic[i]['directions'])
        score = metrics(n_true, n_pred)
        scores = {'noun_recall':score.recall(), 'noun_recall_f':score.recall_freq(),
                  'noun_precision':score.precision(), 'noun_precision_f':score.precision_freq(),
                 }
        dic[i].update(scores)
        score = metrics(v_true, v_pred)
        scores = {'verb_recall':score.recall(), 'verb_recall_f':score.recall_freq(),
                  'verb_precision':score.precision(), 'verb_precision_f':score.precision_freq(),
                 }
        dic[i].update(scores)
        
df2 = pd.DataFrame.from_dict(dic, orient = 'index')
df2[[col for col in df2.columns if 'recall' in col or 'precision' in col]].iloc[ls].mean()

100%|██████████| 55102/55102 [01:52<00:00, 488.83it/s]


noun_recall         0.501185
noun_recall_f       0.442208
noun_precision      0.580863
noun_precision_f    0.501342
verb_recall         0.293824
verb_recall_f       0.279292
verb_precision      0.342315
verb_precision_f    0.299594
dtype: float64

time: 1min 55s


In [ ]:
for i, v in tqdm.tqdm(dic.items()):
    if i in ls:
        ingredients = [sent.split(' ') for sent in v['ingredients']]
        dic[i]['ingred_level2'] = [model.equal_check(ingred) for ingred in ingredients]
        generated_ingredients = [sent.split(' ') for sent in v['generated_ingred']]
        
        dic[i]['generated_ingredients_level2'] = [model.equal_check(instr) for instr in generated_ingredients]
        dic[i]['anymatched_recall'] = anymatched_recall(dic[i]['ingred_level2'], generated_ingredients)
        dic[i]['multiframe_recall'] = multiframe_recall(dic[i]['ingred_level2'], generated_ingredients)
        dic[i]['multiframe_precision'] = multiframe_recall(dic[i]['generated_ingredients_level2'], ingredients)
        
df2 = pd.DataFrame.from_dict(dic, orient = 'index')
df2[['anymatched_recall','multiframe_recall','multiframe_precision']].iloc[ls].mean()

In [ ]:
# batch = 8 top-k sampling (k=3)
filename = '../../to_gpt2/generation_333k/'
ls = load_dir_data(filename)

for i, v in tqdm.tqdm(dic.items()):
    if i in ls:
        ingredients = [sent.split(' ') for sent in v['ingredients']]
        dic[i]['ingred_level2'] = [model.equal_check(ingred) for ingred in ingredients]
        generated_ingredients = [sent.split(' ') for sent in v['generated_ingred']]
        
        dic[i]['generated_ingredients_level2'] = [model.equal_check(instr) for instr in generated_ingredients]
        dic[i]['anymatched_recall'] = anymatched_recall(dic[i]['ingred_level2'], generated_ingredients)
        dic[i]['multiframe_recall'] = multiframe_recall(dic[i]['ingred_level2'], generated_ingredients)
        dic[i]['multiframe_precision'] = multiframe_recall(dic[i]['generated_ingredients_level2'], ingredients)
        
df2 = pd.DataFrame.from_dict(dic, orient = 'index')
df2[['anymatched_recall','multiframe_recall','multiframe_precision']].iloc[ls].mean()

### evlautate wang's model

In [4]:
def read_text_files(filename):
    raw_text = {}
    if os.path.isdir(filename):
        # Directory
        for (dirpath, _, fnames) in os.walk(filename):
            for fname in fnames:
                path = os.path.join(dirpath, fname)
                with open(path, 'r') as fp:
                    raw_text[fname] = fp.read().split('\n')
    return raw_text

filename = '../../to_gpt2/wang/'
raw_text = read_text_files(filename)
len(raw_text['generated_instructions_bert_47.txt']), len(raw_text['gt_instructions.txt'])

(54507, 54507)

time: 155 ms


In [ ]:
dic_evaluate = {}
list_gen, list_gt = raw_text['generated_instructions_bert_47.txt'], raw_text['gt_instructions.txt']
for i, gen in tqdm.tqdm(enumerate(list_gen)):
        n_pred, v_pred = spacy_sentences(gen.split('.'))
        n_true, v_true = spacy_sentences(list_gt[i].split('.'))
        score = metrics(n_true, n_pred)
        scores = {'noun_recall':score.recall(), 'noun_recall_f':score.recall_freq(),
                  'noun_precision':score.precision(), 'noun_precision_f':score.precision_freq(),
                 }
        dic_evaluate[i]=scores
        score = metrics(v_true, v_pred)
        scores = {'verb_recall':score.recall(), 'verb_recall_f':score.recall_freq(),
                  'verb_precision':score.precision(), 'verb_precision_f':score.precision_freq(),
                 }
        dic_evaluate[i].update(scores)
        
df2 = pd.DataFrame.from_dict(dic_evaluate, orient = 'index')
df2[[col for col in df2.columns if 'recall' in col or 'precision' in col]].mean()

### examine error cases in NYtimes parser

In [27]:
dir_save2 = os.path.normpath(dir_HugeFiles+'preprocessing/ingred_20190607.txt')
overwrite = False
with open(dir_save2,'w') as f:
    list_idid = []
    for i, value in dic.items():
        for j, ingred in enumerate(value['ingredients']):
            #assert type(ingred[0]) == str
            #assert ingred != ""
            if not ingred == '' and ' ':
                pair = [i, j, ingred]
                list_idid.append(pair)
                if overwrite:
                    f.write("%s\n" % ingred)
def clean_ny(row):
    '''
    Args:
        row: list of lines
    '''
    return [ele for ele in row if type(ele)==str and len(ele)>0]

time: 1.45 s


use python2 <br>
cd /home/helena/NYtime-parser2/ <br>
python bin/parse-ingredients.py /data/yueliu/RecipeAnalytics_201902/preprocessing/ingred_20190302.txt > results_20190302.txt <br>
python bin/convert-to-json.py results_20190302.txt > results_20190302.json <br>
move the results_20190302.json

In [ ]:
dir_save2 = os.path.normpath('/data/yueliu/RecipeAnalytics_201902/preprocessing/results_20190302.json')
df_idid = pd.DataFrame(list_idid, columns = ['index_1','index_2','ingred'])
parsed = pd.read_json(dir_save2)
df_idid[['input','qty','unit','name','other']] =  parsed[['input','qty','unit','name','other']]
df_parsed = df_idid.groupby('index_1')[['name']].agg(lambda x: list(x)).reset_index(drop = True)
for i, value in dic.items():
    dic[i]['ingred_ny'] = clean_ny(df_parsed.iloc[i]['name'])

In [76]:
df = df_idid[df_idid['name'].isna()]
display(df.head())
display(df[df['ingred'].apply(lambda x: '(' not in x)].head())

,index_1,index_2,ingred,input,qty,unit,name,other
3,0,3,1 ( 16 ounce ) package all beef hot dogs,1 (16 ounce) package all beef hot dogs,1,NaN,NaN,NaN
48,6,5,salt to taste,salt to taste,salt,NaN,NaN,NaN
56,6,13,"1 ( 4 ounce ) package white anchovies , washed","1 (4 ounce) package white anchovies, washed",1,NaN,NaN,NaN
63,6,20,"1 ( 4 ounce ) package white anchovies , washed","1 (4 ounce) package white anchovies, washed",1,NaN,NaN,NaN
64,6,21,salt to taste,salt to taste,salt,NaN,NaN,NaN


,index_1,index_2,ingred,input,qty,unit,name,other
48,6,5,salt to taste,salt to taste,salt,NaN,NaN,NaN
64,6,21,salt to taste,salt to taste,salt,NaN,NaN,NaN
110,11,13,2 cups half and half,2 cups half and half,2,cup,NaN,NaN
275,30,0,"1 avocado peeled , pitted , and cut into chunks","1 avocado peeled, pitted, and cut into chunks",1,NaN,NaN,","
315,35,4,salt to taste,salt to taste,salt,NaN,NaN,NaN


time: 141 ms


In [ ]:
### TODO 
### delete the words within braces
### use NY-times parser
### manually fix error cases such as 


In [8]:
p = inflect.engine()
class closest:
    def __init__(self, series):
        find_ngram = series.apply(lambda x: ' ' in x)
        self.unigram_ingredients, self.ngram_ingredients = series[~find_ngram], series[find_ngram]
        # self.unigram_ingredients_pluraled = [p.plural(ingr) for ingr in self.unigram_ingredients]
    
    def ngram(self, listofline, pluraled_listofline):
        answer = []
        for ngram in self.ngram_ingredients.values:
            n_list= ngram.split(' ')
            intersection = set(listofline) & set(n_list)
            if len(intersection) >= 2 and len(intersection) == len(n_list):
                answer.append(ngram)
            else:
                # repeat
                intersection = (set(pluraled_listofline) | set(intersection)) & set(n_list)
                if len(intersection) >= 2 and len(intersection) == len(n_list):
                    answer.append(ngram)
        return set(answer)
    
    def unigram(self, listofline, pluraled_listofline):
        first =  set(listofline) & set(self.unigram_ingredients)
        if first:
            return first
        second = set(pluraled_listofline) & set(self.unigram_ingredients)
        if second:
            return second
        
    def check(self, listofline):
        '''
        assign higher priority to n-grams over uni-gram
        '''
        pluraled_listofline = [p.plural(word) for word in listofline]
        ngram = self.ngram(listofline, pluraled_listofline)
        if ngram:
            return ngram
        unigram = self.unigram(listofline, pluraled_listofline)
        if unigram:
            return unigram
        
    def equal_check(self, listofline):
        '''
        equally care about unigram and n gram
        '''
        pluraled_listofline = [p.plural(word) for word in listofline]
        ngram = self.ngram(listofline, pluraled_listofline)
        unigram = self.unigram(listofline, pluraled_listofline)
        if ngram and unigram:
            return ngram | unigram
        if ngram:
            return ngram
        if unigram:
            return unigram

time: 29.3 ms


In [ ]:
class closest_lighter(closest):
    def __init__(self, set_grams):
        self.unigram_ingredients = pd.Series([x for x in set_grams if ' ' not in x])
        self.ngram_ingredients = pd.Series([x for x in set_grams if ' ' in x])

In [9]:
# prepare the ingredient detection algorithm
model = closest(df[2])
def multiframe_recall(ingred_level2, directions):
    '''
    ingred_level2: dic[i]['ingred_level2']; list of set; ground truth;
    directions: dic[i]['directions']; list of words; prediciton
    denominator: number of identified n-grams
    '''
    try:
        # recall 
        true_flatten = set.union(*[line for line in ingred_level2 if line])
        model_lighter = closest_lighter(true_flatten)
        direct_level2 = [model_lighter.equal_check(instr) for instr in directions]
        pred_flatten = set.union(*[line for line in direct_level2 if line])
        true_positive = true_flatten & pred_flatten

        recall = len(true_positive)/len(true_flatten)
        #precision = len(true_positive)/len(pred_flatten)
        #f1 = 2* recall * precision / (recall + precision)
        return recall #, precision, f1
    except TypeError:
        return 0
    

def anymatched_recall(ingred_level2, directions):
    '''
    ingred_level2: dic[i]['ingred_level2']; list of set; ground truth;
    directions: dic[i]['directions']; list of words; prediciton
    denominator: number of ingredients
    '''
    try:
        # recall 
        true_flatten = set.union(*[line for line in ingred_level2 if line])
        model_lighter = closest_lighter(true_flatten)
        direct_level2 = [model_lighter.equal_check(instr) for instr in directions]
        pred_flatten = set.union(*[line for line in direct_level2 if line])
        true_positive = true_flatten & pred_flatten

        recall = len(true_positive)/len(true_flatten)
        return np.mean([1 if line & true_positive else 0 for line in ingred_level2 if line])
    except TypeError:
        return 0

time: 34.4 ms


In [11]:
filename = '../../to_gpt2/generation_308k/'
def load_dir_data(filename):
    ls = []
    if os.path.isdir(filename):
        # Directory
        for (dirpath, _, fnames) in os.walk(filename):
            for fname in fnames:
                path = os.path.join(dirpath, fname)
                with open(path, 'r') as fp:
                    raw_text = fp.read()
                # if it contain instr
                if fname[-5] == 'd':
                    dic[int(fname[:-5])]['generated_instr'] = [raw_text]
                # if it contain ingred
                if fname[-5] == 'i':
                    dic[int(fname[:-5])]['generated_ingred'] = raw_text.split('$')
                # if it contain name
                if fname[-5] == 'n':
                    dic[int(fname[:-5])]['generated_name'] = raw_text
                ls.append(int(fname[:-5]))
    return sorted(list(set(ls)))
ls = load_dir_data(filename)

time: 103 ms


In [67]:
for i, v in tqdm.tqdm(dic.items()):
    if i in ls:
        ingredients = [sent.split(' ') for sent in v['ingredients']]
        dic[i]['ingred_level2'] = [model.equal_check(ingred) for ingred in ingredients]
        generated_ingredients = [sent.split(' ') for sent in v['generated_ingred']]
        
        dic[i]['generated_ingredients_level2'] = [model.equal_check(instr) for instr in generated_ingredients]
        dic[i]['anymatched_recall'] = anymatched_recall(dic[i]['ingred_level2'], generated_ingredients)
        dic[i]['multiframe_recall'] = multiframe_recall(dic[i]['ingred_level2'], generated_ingredients)
        dic[i]['multiframe_precision'] = multiframe_recall(dic[i]['generated_ingredients_level2'], ingredients)
        
df2 = pd.DataFrame.from_dict(dic, orient = 'index')
df2[['anymatched_recall','multiframe_recall','multiframe_precision']].iloc[ls].mean()

100%|██████████| 55102/55102 [00:45<00:00, 1200.81it/s]


anymatched_recall       0.498159
multiframe_recall       0.427480
multiframe_precision    0.304958
dtype: float64

time: 48.5 s
